## localize anomaly by detecting  generated abnormal pixels

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils
import os
import tensorflow as tf

# Adjusted threshold for overall frame anomaly detection
# IMPORTANT: This threshold will likely need re-tuning after changing mean_squared_loss
ANOMALY_THRESHOLD = 0.00435 # A more typical unscaled MSE threshold for normalized data

# Threshold for identifying anomalous regions within a frame (for visualization)
# This value will likely need to be tuned based on your model's performance and data.
# It represents a pixel-wise squared error value.
ANOMALY_REGION_PIXEL_THRESHOLD = 0.200 # Example value, tune this!

model_dir = "../models/model_e50_b4.h5" # Path to the trained Keras model
video_path = "D:\\Research Dataset\\Surveillance\\Test\\testing_video.mp4"


# Ensure output folders exist
if not os.path.exists("output_frames"):
    os.makedirs("output_frames")
if not os.path.exists("reconstructed_frames"):
    os.makedirs("reconstructed_frames")
if not os.path.exists("anomaly_maps"): # New folder for error maps
    os.makedirs("anomaly_maps")
frame_count_processed = 0
anomaly_count = 0

def mean_squared_loss(x1, x2):
    """
    Calculates the true Mean Squared Error (MSE) between two arrays.
    This function is now consistent with tf.keras.losses.mean_squared_error.
    Returns the MSE.
    """
    # Ensure inputs are numpy arrays for consistent calculation,
    # as 'output' from model.predict is typically a numpy array.
    # If x1 or x2 were TensorFlow tensors, .numpy() would convert them.
    x1_np = x1.numpy() if tf.is_tensor(x1) else x1
    x2_np = x2.numpy() if tf.is_tensor(x2) else x2

    # Flatten the arrays to compute element-wise difference
    difference = x1_np.flatten() - x2_np.flatten()
    sq_difference = difference ** 2
    mse = np.mean(sq_difference) # Correct MSE calculation
    return mse

def combined_loss(y_true, y_pred):
    """
    Custom combined loss function for the Keras model.
    Assumes mean squared error is the primary component.
    This function is used during model loading/compilation.
    """
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return mse_loss

def preprocess_frame_for_new_model(frame):
    """
    Preprocess frame for the new model architecture.
    The new model expects normalized inputs (e.g., in [-1, 1] range).
    """
    frame_resized = cv2.resize(frame, (232, 232), interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    gray = gray.astype(np.float32) / 255.0 # Normalize to [0, 1]
    gray = (gray - 0.5) * 2 # Scale to [-1, 1]
    return gray

# Load the trained Keras model
custom_objects = {'combined_loss': combined_loss}
try:
    model = load_model(model_dir, custom_objects=custom_objects)
    print("Frame-by-frame model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure 'model.h5' is in the '../models/' directory and the custom loss function is defined.")
    exit()

# Open the video file for processing
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit()

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {length}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or issue reading frame.")
        break

    frame_count_processed += 1

    # Resize original frame for consistent display
    original_frame_color = imutils.resize(frame, width=700, height=600)

    # Preprocess frame for the model
    processed_frame = preprocess_frame_for_new_model(frame)
    # Add batch and channel dimensions for model input (batch_size, height, width, channels)
    model_input_frame = np.expand_dims(np.expand_dims(processed_frame, axis=0), axis=-1)

    # Get model prediction (reconstructed frame)
    output = model.predict(model_input_frame)

    # Extract the 2D reconstructed frame from the model output
    reconstructed_frame_2d = output[0, :, :, 0]

    # --- Save Model Prediction (Individual Reconstructed Frame as Image) ---
    # Normalize reconstructed frame to [0, 255] for display and saving
    # If the model output is in [-1, 1], map it back to [0, 255]
    reconstructed_frame_display = ((reconstructed_frame_2d + 1) / 2 * 255).astype(np.uint8)
    # Resize reconstructed frame to match the display size of the original frame
    reconstructed_frame_display = cv2.resize(
        reconstructed_frame_display, (original_frame_color.shape[1], original_frame_color.shape[0])
    )
    # Save the reconstructed frame
    cv2.imwrite(f"reconstructed_frames/frame_{frame_count_processed:05d}_reconstructed.jpg", reconstructed_frame_display)
    # --- End Save Model Prediction (Image) ---
    # Calculate the overall reconstruction loss (MSE) for the frame
    loss = mean_squared_loss(model_input_frame, output)
    # Removed loss_scaled as it's not needed with the corrected MSE and threshold
    loss_display = f"{loss:.5f}"
    print(f"Frame {frame_count_processed} Loss: {loss_display}")

    display_frame = original_frame_color.copy()

    status_text = f"Normal: {loss_display}"
    color = (0, 255, 0) # Green for normal

    # Check for overall frame anomaly
    if loss > ANOMALY_THRESHOLD:
        status_text = f"Anomaly: {loss_display}"
        color = (0, 0, 255) # Red for abnormal
        anomaly_count += 1 # Increment anomaly count only when overall frame is abnormal

        # --- Anomaly Localization (only if the frame is considered abnormal) ---
        # Calculate the element-wise squared difference between original and reconstructed
        # Ensure both are in the same range (e.g., [-1, 1]) for this calculation
        squared_diff_map = (processed_frame - reconstructed_frame_2d)**2

        # Normalize the squared difference map to [0, 255] for visualization
        # This makes higher differences appear brighter
        error_map_display = cv2.normalize(
            squared_diff_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U
        )
        # Resize the error map to the display frame's dimensions
        error_map_display_resized = cv2.resize(
            error_map_display, (original_frame_color.shape[1], original_frame_color.shape[0]),
            interpolation=cv2.INTER_LINEAR
        )
        cv2.imwrite(f"anomaly_maps/frame_{frame_count_processed:05d}_error_map.jpg", error_map_display_resized)
        
        # Create a binary mask of high error regions
        # Scale ANOMALY_REGION_PIXEL_THRESHOLD to 255 for cv2.threshold if squared_diff_map is normalized
        # Or, directly threshold the squared_diff_map values.
        # Ensure the input to threshold is 8-bit unsigned integer
        _, binary_error_mask = cv2.threshold(
            (squared_diff_map * 255).astype(np.uint8), # Scale to 0-255 for thresholding
            (ANOMALY_REGION_PIXEL_THRESHOLD * 255), # Scale threshold too
            255,
            cv2.THRESH_BINARY
        )

        # Resize the binary mask to the display frame's dimensions
        binary_error_mask_resized = cv2.resize(
            binary_error_mask, (original_frame_color.shape[1], original_frame_color.shape[0]),
            interpolation=cv2.INTER_NEAREST # Use nearest for binary masks
        )

        # Find contours in the binary mask
        contours, _ = cv2.findContours(
            binary_error_mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        # Draw bounding boxes around significant anomalous regions
        for contour in contours:
            # Filter out small noise contours
            if cv2.contourArea(contour) > 50: # Adjust this area threshold as needed
                x, y, w, h = cv2.boundingRect(contour)

                # Create a semi-transparent red overlay for the bounding box
                overlay = display_frame.copy()
                cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 0, 255), -1) # Filled red rectangle
                alpha = 0.50 # Transparency factor. 0.0 is fully transparent, 1.0 is fully opaque.
                display_frame = cv2.addWeighted(overlay, alpha, display_frame, 1 - alpha, 0)

                # Removed the line that draws the red border, leaving only the shade
                # cv2.rectangle(display_frame, (x, y), (x + w, y + h), (0, 0, 255), 2) # Red border
        # --- End Anomaly Localization ---

    # Put status text on the display frame
    cv2.putText(display_frame, status_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Save the annotated frame to output_frames
    if status_text.startswith("Anomaly"):
        cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}.jpg", display_frame)
    else:
        cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}.jpg", display_frame)

    # Display the frame
    cv2.imshow("Anomaly Detection", display_frame)
    # Wait for a key press (1ms delay), exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Print progress every 100 frames
    if frame_count_processed % 100 == 0:
        print(f"Processed {frame_count_processed} frames, found {anomaly_count} anomalies so far.")

# Release video capture and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()

print(f"\nProcessing complete!")
print(f"Total frames processed: {frame_count_processed}")
print(f"Total anomalies detected: {anomaly_count}")
if frame_count_processed > 0:
    anomaly_rate = (anomaly_count / frame_count_processed * 100)
    print(f"Anomaly rate: {anomaly_rate:.2f}%")
else:
    print("No frames were processed, cannot calculate anomaly rate.")


Frame-by-frame model loaded successfully.
Total frames in video: 200
1/1 [==============================] - 9s 9s/step
Frame 1 Loss: 0.00413
1/1 [==============================] - 0s 21ms/step
Frame 2 Loss: 0.00415
1/1 [==============================] - 0s 18ms/step
Frame 3 Loss: 0.00417
1/1 [==============================] - 0s 18ms/step
Frame 4 Loss: 0.00417
1/1 [==============================] - 0s 17ms/step
Frame 5 Loss: 0.00419
1/1 [==============================] - 0s 20ms/step
Frame 6 Loss: 0.00416
1/1 [==============================] - 0s 18ms/step
Frame 7 Loss: 0.00416
1/1 [==============================] - 0s 21ms/step
Frame 8 Loss: 0.00422
1/1 [==============================] - 0s 18ms/step
Frame 9 Loss: 0.00432
1/1 [==============================] - 0s 18ms/step
Frame 10 Loss: 0.00418
1/1 [==============================] - 0s 19ms/step
Frame 11 Loss: 0.00420
1/1 [==============================] - 0s 18ms/step
Frame 12 Loss: 0.00416
1/1 [==============================] - 0s 

## Output with graph analysis

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils
import os
import tensorflow as tf
import matplotlib.pyplot as plt

# Adjusted threshold for overall frame anomaly detection
ANOMALY_THRESHOLD = 0.00435
ANOMALY_REGION_PIXEL_THRESHOLD = 0.200

# Define model and video paths
model_dir = "../models/model_e50_b4.h5"
video_path = "D:\\Research Dataset\\Surveillance\\Test\\testing_video.mp4"
output_video_path = "loss_graph_video.mp4" # Path to save the plot video
plot_frame_dir = "plot_frames" # Directory to save individual plot frames

# Create output directories if they don't exist
if not os.path.exists("output_frames"):
    os.makedirs("output_frames")
if not os.path.exists("reconstructed_frames"):
    os.makedirs("reconstructed_frames")
if not os.path.exists("anomaly_maps"):
    os.makedirs("anomaly_maps")
if not os.path.exists(plot_frame_dir):
    os.makedirs(plot_frame_dir)

# Lists to store frame loss data and frame numbers
frame_losses = []
frame_numbers = []
frame_count_processed = 0
anomaly_count = 0


# Function to calculate Mean Squared Error (MSE) loss
def mean_squared_loss(x1, x2):
    x1_np = x1.numpy() if tf.is_tensor(x1) else x1
    x2_np = x2.numpy() if tf.is_tensor(x2) else x2
    difference = x1_np.flatten() - x2_np.flatten()
    sq_difference = difference ** 2
    mse = np.mean(sq_difference)
    return mse

# Custom loss function for Keras model loading
def combined_loss(y_true, y_pred):
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return mse_loss

# Function to preprocess frames for the model
def preprocess_frame_for_new_model(frame):
    frame_resized = cv2.resize(frame, (232, 232), interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    gray = gray.astype(np.float32) / 255.0
    gray = (gray - 0.5) * 2
    return gray

# Load the pre-trained Keras model
custom_objects = {'combined_loss': combined_loss}
try:
    model = load_model(model_dir, custom_objects=custom_objects)
    print("Frame-by-frame model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure 'model.h5' is in the '../models/' directory and the custom loss function is defined.")
    exit()

# Open the video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit()
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {length}")


# Initialize the plot
fig, ax = plt.subplots()
line, = ax.plot(frame_numbers, frame_losses)
ax.set_xlabel('Frame Number')
ax.set_ylabel('Mean Squared Error (MSE)')
ax.set_title('Real-time Reconstruction Loss')
ax.grid(True)

# Set initial fixed x-axis limits
initial_x_limit = length
ax.set_xlim(0, initial_x_limit)
# Set initial fixed y-axis limits from 0 to 2
ax.set_ylim(0, 0.030) ## I may need to adjust it

# Main loop for frame processing
while True:
    ret, raw_frame = cap.read()
    if not ret:
        print("End of video.")
        break

    frame_count_processed += 1
    original_frame_color = imutils.resize(raw_frame, width=700, height=600)
    processed_frame = preprocess_frame_for_new_model(raw_frame)
    model_input_frame = np.expand_dims(np.expand_dims(processed_frame, axis=0), axis=-1)
    output = model.predict(model_input_frame)
    reconstructed_frame_2d = output[0, :, :, 0]
    reconstructed_frame_display = ((reconstructed_frame_2d + 1) / 2 * 255).astype(np.uint8)
    reconstructed_frame_display = cv2.resize(
        reconstructed_frame_display, (original_frame_color.shape[1], original_frame_color.shape[0])
    )
    cv2.imwrite(f"reconstructed_frames/frame_{frame_count_processed:05d}_reconstructed.jpg", reconstructed_frame_display)

    loss = mean_squared_loss(model_input_frame, output)
    loss_display = f"{loss:.5f}"
    print(f"Frame {frame_count_processed} Loss: {loss_display}")

    frame_losses.append(loss)
    frame_numbers.append(frame_count_processed)

    # Update the plot data
    line.set_xdata(frame_numbers)
    line.set_ydata(frame_losses)

    # Dynamically adjust x-axis limits for scrolling effect
    if frame_count_processed > initial_x_limit:
        ax.set_xlim(frame_count_processed - initial_x_limit, frame_count_processed)
    else:
        ax.set_xlim(0, initial_x_limit)

    # Redraw the canvas and save the plot frame
    fig.canvas.draw()
    plot_frame_filename = os.path.join(plot_frame_dir, f"plot_frame_{frame_count_processed:05d}.png")
    fig.savefig(plot_frame_filename) # Save the plot as an image

    display_frame = original_frame_color.copy()
    status_text = f"Normal: {loss_display}"
    color = (0, 255, 0)

    if loss > ANOMALY_THRESHOLD:
        status_text = f"Anomaly: {loss_display}"
        color = (0, 0, 255)
        anomaly_count += 1

        # --- Anomaly Localization with Overlay ---
        squared_diff_map = (processed_frame - reconstructed_frame_2d)**2
        # Normalize the squared difference map to [0, 255] for thresholding
        scaled_diff_map = (squared_diff_map * 255).astype(np.uint8)
        _, binary_error_mask = cv2.threshold(
            scaled_diff_map,
            int(ANOMALY_REGION_PIXEL_THRESHOLD * 255),
            255,
            cv2.THRESH_BINARY
        )
        binary_error_mask_resized = cv2.resize(
            binary_error_mask, (original_frame_color.shape[1], original_frame_color.shape[0]),
            interpolation=cv2.INTER_NEAREST
        )
        contours, _ = cv2.findContours(
            binary_error_mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        for contour in contours:
            if cv2.contourArea(contour) > 50:
                x, y, w, h = cv2.boundingRect(contour)
                overlay = display_frame.copy()
                cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 0, 255), -1)
                alpha = 0.50
                display_frame = cv2.addWeighted(overlay, alpha, display_frame, 1 - alpha, 0)
        # --- End Anomaly Localization ---

    cv2.putText(display_frame, status_text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    cv2.imwrite(f"output_frames/frame_{frame_count_processed:05d}.jpg", display_frame)
    cv2.imshow("Anomaly Detection", display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Quitting video processing.")
        break

# Release video capture and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()
plt.close(fig) # Close the Matplotlib figure

print(f"\nProcessing complete!")
print(f"Total frames processed: {frame_count_processed}")
print(f"Total anomalies detected: {anomaly_count}")
if frame_count_processed > 0:
    anomaly_rate = (anomaly_count / frame_count_processed * 100)
    print(f"Anomaly rate: {anomaly_rate:.2f}%")
else:
    print("No frames were processed, cannot calculate anomaly rate.")

# Save the final loss data to a CSV file
loss_data = np.array([frame_numbers, frame_losses]).T
np.savetxt("loss_data_final.csv", loss_data, delimiter=",", header="frame_number,loss", comments="")
print("Final loss information saved to loss_data_final.csv")
## 54.8 sec without trt model

Frame-by-frame model loaded successfully.
Total frames in video: 200
1/1 [==============================] - 7s 7s/step
Frame 1 Loss: 0.00275
1/1 [==============================] - 0s 17ms/step
Frame 2 Loss: 0.00274
1/1 [==============================] - 0s 17ms/step
Frame 3 Loss: 0.00278
1/1 [==============================] - 0s 18ms/step
Frame 4 Loss: 0.00272
1/1 [==============================] - 0s 19ms/step
Frame 5 Loss: 0.00272
1/1 [==============================] - 0s 19ms/step
Frame 6 Loss: 0.00278
1/1 [==============================] - 0s 17ms/step
Frame 7 Loss: 0.00280
1/1 [==============================] - 0s 20ms/step
Frame 8 Loss: 0.00282
1/1 [==============================] - 0s 18ms/step
Frame 9 Loss: 0.00287
1/1 [==============================] - 0s 18ms/step
Frame 10 Loss: 0.00281
1/1 [==============================] - 0s 18ms/step
Frame 11 Loss: 0.00278
1/1 [==============================] - 0s 19ms/step
Frame 12 Loss: 0.00273
1/1 [==============================] - 0s 